In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page='

data = requests.get(URL).text

In [3]:
type(data)

str

In [4]:
with open('./data/mercadolivreoferdia.html','w', encoding='utf-8') as f:
    f.write(data)

In [5]:
def get_items_title(doc):
    """
        Return the title of products from tags
    """
    title_tags = doc.find_all('p',class_='promotion-item__title')
    titles = []

    for tags in title_tags:
        titles.append(tags.text)

    return titles

In [6]:
# BUG - Considering the previous price as a real price, duplicating the array of data.
def get_items_price(doc):
    """
        Return the price of products from tags
    """
    prices = []

    price_parent_tags = doc.find_all('span',class_='andes-money-amount andes-money-amount--cents-superscript')
    
    for child in price_parent_tags: 
        prices.append(child.find('span',class_='andes-money-amount__fraction').text)

    return prices

In [7]:
def get_doc(url):
    """
        Return the html parsed instance.
    """
    time.sleep(1)
    response =  requests.get(url)
    doc = BeautifulSoup(response.text,'html.parser')

    if response.status_code != 200:
        raise Exception("Failure to load the page!") 

    return doc

In [8]:
def scrape_multiple_pages(n):
    titles, prices = [] , [] 

    for page in range(1, n+1):
        doc = get_doc(URL + str(page))
        titles.extend(get_items_title(doc))
        prices.extend(get_items_price(doc))

    items = {
        'TITLE': titles,
        'PRICE': prices
    }

    # return pd.DataFrame.from_dict(items,orient='index').transpose()
    return pd.DataFrame(items)

In [9]:
df = scrape_multiple_pages(10)

df.head()

,TITLE,PRICE
0,Samsung Galaxy A15 4G Dual SIM 128 GB Azul cla...,879
1,Console Sony PlayStation 5 Standard Edition 825GB,3.543
2,Kit Com 5 Camisetas Masculinas Básicas Hering,127
3,Smart Tv LG 50 Led 4k Uhd Wi-fi Bluetooth Hdr1...,2.129
4,Gel De Limpeza Facial Effaclar Concentrado 300...,65


In [10]:
df.to_csv("./data/mercado_livre.csv",mode='w',index=False)